In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import datetime

In [2]:
file = '../data/new_5sta(DNOWS)_3com(NO2,PM10K,O3)_year2010_TemporalDummies_Ox_holidays_meteo.csv'
               

AREAS = ['D_', 'N_', 'O_', 'S_', 'W_']

factors = ['NO2', 'PM10K', 'O3', 'Ox']

col_names = [['DonBosco_', 'D_'], ['Nord_', 'N_'], 
             ['Ost_', 'O_'], ['Sud_', 'S_'], ['West_', 'W_'],              
            ]

add_names = {
    'D_': 'DonBosco_',
    'N_': 'Nord_',
    'O_': 'Ost_',
    'W_': 'West_',
    'S_': 'Sud_'
} 

features2=['weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday']

features3 = ['season_fall', 'season_spring',
             'season_summer', 'season_winter'
]    


holidays = ['holiday', 'holiday_school']

In [3]:
def cut_off(df_o, percent):
    df_o = df_o.copy()
    cols = [k for k in [i+j for j in factors for i in add_names.keys()]
            if k in df_o.columns
           ]
    # print(cols)
    targets = df_o[cols]
    quantiles = targets.quantile(percent)
    for c in cols:
        df_o[c] = [i if i < quantiles[c] else quantiles[c] 
                     for i in df_o[c]]
    return df_o

In [4]:
def ml(df_o, factor, params,
        cutoff,
        s1='2020-01-03', e1='2020-03-10', 
       features2=features2, features3=features3, others=[],
       train_date=None, init_points=10, n_iter=5
      ):
    results = pd.DataFrame(index=['RMSE', 'MAE'])
    for a, b in col_names:
        params_gbm = params.copy()
        print(a, b)
        if b+factor not in df_o.columns: continue
        cols = [i for i in df_o.columns if a in i]
            
        cols += features2+features3+others
        label_cols = [i for i in df_o.columns if b in i]

        df = df_o[cols + label_cols + ['year']]
        check_NA = df.isna().sum()
        if check_NA[check_NA>0].shape[0]>0:
            # print(f'check NA\n{check_NA}')
            df = df.dropna(how='any')
            
        if train_date is None:    
            train = df[df['year']<2020]
        else:
            train = df.loc[:str(datetime.date.fromisoformat(s1) - datetime.timedelta(days=1))]
        test = df.loc[s1:e1]
        
        # cutoffs
        train = cut_off(train, cutoff)
        
        X_train = train[cols]
        y_train = train[b+factor]

        X_test = test[cols]
        y_test = test[b+factor]

        def gbm_cl_bo(max_depth,   
                        n_estimators
                        ):
            params_gbm = {}
            params_gbm['max_depth'] = round(max_depth)
            params_gbm['n_estimators'] = round(n_estimators)

            scores = cross_val_score(RandomForestRegressor(                                            
                                            **params_gbm),
                                    X_train, y_train, cv=5).mean()
            score = scores.mean()
            return score

        gbm_bo = BayesianOptimization(gbm_cl_bo, params_gbm,
                                    random_state=111
                                    )
        gbm_bo.maximize(init_points=init_points, n_iter=n_iter)

        params_gbm = gbm_bo.max['params']
        params_gbm['max_depth'] = round(params_gbm['max_depth'])
        params_gbm['n_estimators'] = round(params_gbm['n_estimators'])
        print(params_gbm)
        
        clf = RandomForestRegressor(**params_gbm)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred = pd.Series(y_pred, index=y_test.index)
        mse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        results[b+factor] = [mse, mae]   
    
    return results

In [5]:
df = pd.DataFrame()
params_gbm ={
    'max_depth':(1, 10),
    'n_estimators':(30, 500)
}

percent = 0.90

def my_f(percent=percent, df=df):
    print(f'percent: {percent}')
    df_o = pd.read_csv(file, index_col=0, parse_dates=True)
    months = [c for c in df_o.columns if 'month' in c]
    # f0
    ################################################
    tmp = pd.DataFrame()
    method = 'rf_base_'
    for factor in factors:
        print(factor, method)

        r = ml(df_o=df_o, factor=factor, 
            features2=[], features3=[],
            
            cutoff=percent,
            params=params_gbm, 
            
            train_date=None,
            init_points=20, n_iter=5,                            
            )
        tmp = pd.concat([tmp, r], axis=1)

    tmp.index = tmp.index + '_' + method
    df = pd.concat([df, tmp])

    # f1
    ################################################
    tmp = pd.DataFrame()
    method = 'rf_1_'
    for factor in factors:
        print(factor, method)
        r = ml(df_o=df_o, factor=factor, 
            
            cutoff=percent,
            params=params_gbm, 
            
            train_date=None,
            init_points=20, n_iter=5
            )
        tmp = pd.concat([tmp, r], axis=1)
    tmp.index = tmp.index + '_' + method
    df = pd.concat([df, tmp])

    # f2
    ################################################
    tmp = pd.DataFrame()
    method = 'rf_2_'
    for factor in factors:
        print(factor, method)
        r = ml(df_o=df_o, factor=factor, 
            others=months+holidays, 
            
            cutoff=percent,
            params=params_gbm, 
            
            train_date=None,
            init_points=20, n_iter=5
            )
        tmp = pd.concat([tmp, r], axis=1)
    tmp.index = tmp.index + '_' + method
    df = pd.concat([df, tmp])

    # f3
    ################################################
    weekday_convert = {
        'weekday_Sunday': 0,    
        'weekday_Monday': 1,
        'weekday_Tuesday': 2, 
        'weekday_Wednesday': 3,
        'weekday_Thursday': 4,
        'weekday_Friday': 5,
        'weekday_Saturday': 6
    }
    for c in features2:
        df_o[c] = [weekday_convert[c] if i else i for i in df_o[c]]
    df_o['dayoftheweek'] = df_o[features2].sum(axis=1)


    season_convert = {
        'season_spring': 1,
        'season_summer': 2,
        'season_fall': 3,
        'season_winter': 4
    }

    for c in season_convert.keys():
        df_o[c] = [season_convert[c] if i else i for i in df_o[c]]
    df_o['season'] = df_o[season_convert.keys()].sum(axis=1)


    df_o['cross1'] = df_o['season']*df_o['dayoftheweek']
    df_o['cross2'] = df_o['season']**2 + df_o['dayoftheweek']**2
    df_o['cross3'] = df_o['season']**2
    df_o['cross4'] = df_o['dayoftheweek']**2
    crosses = ['cross1', 'cross2', 'cross3', 'cross4']

    tmp = pd.DataFrame()
    method = 'rf_3_'
    for factor in factors:
        print(factor, method)
        r = ml(df_o=df_o, factor=factor, 
            others=crosses+months+holidays,
            
            cutoff=percent,
            params=params_gbm, 
            
            train_date=None,
            init_points=20, n_iter=5
            )
        tmp = pd.concat([tmp, r], axis=1)
    tmp.index = tmp.index + '_' + method
    df = pd.concat([df, tmp])

    # f4
    ################################################
    df_o['cross5'] = df_o['season']**3
    df_o['cross6'] = df_o['dayoftheweek']**3
    crosses2 = crosses + ['cross5', 'cross6']

    tmp = pd.DataFrame()
    method = 'rf_4_'
    for factor in factors:
        print(factor, method)
        r = ml(df_o=df_o, factor=factor, 
            others=crosses2,
            
            cutoff=percent,
            params=params_gbm, 
            
            train_date=None,
            init_points=20, n_iter=5
            )
        tmp = pd.concat([tmp, r], axis=1)
    tmp.index = tmp.index + '_' + method
    df = pd.concat([df, tmp])


    # f5
    ################################################
    features=['Temp', 'RH', 'Pressure', 
            'Winddirection', 'Windspeed', 'Precip']
    cols = [c for c in [j+i for j in add_names.values() \
                        for i in features] if c in df_o.columns]
    for c in cols:
        df_o[c+'_2'] = df_o[c]**2
    df_o[[c+'_2' for c in cols]]

    tmp = pd.DataFrame()
    method = 'rf_5_'
    for factor in factors:
        print(factor, method)
        r = ml(df_o=df_o, factor=factor, 
            others=crosses2,
            
            cutoff=percent,
            params=params_gbm, 
            
            train_date=None,
            init_points=20, n_iter=5
            )                    
        tmp = pd.concat([tmp, r], axis=1)
    tmp.index = tmp.index + '_' + method
    df = pd.concat([df, tmp])

    # f6
    ################################################
    tmp = pd.DataFrame()
    method = 'rf_6_'
    for factor in factors:
        print(factor, method)
        r = ml(df_o=df_o, factor=factor, 
            others=['season', 'dayoftheweek'],
            
            cutoff=percent,
            params=params_gbm, 
            
            train_date=None,
            init_points=20, n_iter=5
            )
        tmp = pd.concat([tmp, r], axis=1)
    tmp.index = tmp.index + '_' + method
    df = pd.concat([df, tmp])

    return df

In [6]:
df = my_f()
df.to_csv(f'../output_intermediate/ML_Optimization/rf/rf_s1_{percent}.csv')

percent: 0.9
NO2 rf_base_
DonBosco_ D_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.2084   |  6.51     |  109.5    |
|  2        |  0.217    |  4.925    |  391.6    |
|  3        |  0.2165   |  3.658    |  100.1    |
|  4        |  0.142    |  1.202    |  227.5    |
|  5        |  0.2056   |  3.148    |  188.7    |
|  6        |  0.1855   |  9.916    |  141.7    |
|  7        |  0.1786   |  1.731    |  344.7    |
|  8        |  0.2092   |  6.591    |  158.9    |
|  9        |  0.2166   |  5.196    |  85.63    |
|  10       |  0.1778   |  1.666    |  453.4    |
|  11       |  0.2026   |  8.146    |  425.1    |
|  12       |  0.2025   |  8.337    |  495.7    |
|  13       |  0.2122   |  6.195    |  412.5    |
|  14       |  0.2145   |  4.792    |  42.9     |
|  15       |  0.2157   |  5.087    |  79.5     |
|  16       |  0.2022   |  8.355    |  357.9    |
|  17       |  0.2136   |  6.088    |  158.9    |
|  18      

|  17       |  0.609    |  6.088    |  158.9    |
|  18       |  0.6074   |  9.986    |  94.88    |
|  19       |  0.6134   |  6.539    |  258.0    |
|  20       |  0.597    |  4.645    |  372.1    |
|  21       |  0.6082   |  10.0     |  276.5    |
|  22       |  0.3482   |  1.0      |  125.1    |
|  23       |  0.3477   |  1.0      |  268.7    |
|  24       |  0.6073   |  9.949    |  150.7    |
|  25       |  0.6071   |  9.755    |  250.1    |
{'max_depth': 8, 'n_estimators': 496}
West_ W_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.6013   |  6.51     |  109.5    |
|  2        |  0.5874   |  4.925    |  391.6    |
|  3        |  0.5674   |  3.658    |  100.1    |
|  4        |  0.3305   |  1.202    |  227.5    |
|  5        |  0.532    |  3.148    |  188.7    |
|  6        |  0.5968   |  9.916    |  141.7    |
|  7        |  0.4661   |  1.731    |  344.7    |
|  8        |  0.6024   |  6.591    |  158.9    |
|  

|  7        |  0.1314   |  1.731    |  344.7    |
|  8        |  0.1926   |  6.591    |  158.9    |
|  9        |  0.204    |  5.196    |  85.63    |
|  10       |  0.1279   |  1.666    |  453.4    |
|  11       |  0.189    |  8.146    |  425.1    |
|  12       |  0.1877   |  8.337    |  495.7    |
|  13       |  0.2048   |  6.195    |  412.5    |
|  14       |  0.2061   |  4.792    |  42.9     |
|  15       |  0.2012   |  5.087    |  79.5     |
|  16       |  0.1844   |  8.355    |  357.9    |
|  17       |  0.194    |  6.088    |  158.9    |
|  18       |  0.1771   |  9.986    |  94.88    |
|  19       |  0.1991   |  6.539    |  258.0    |
|  20       |  0.2048   |  4.645    |  372.1    |
|  21       |  0.171    |  10.0     |  282.1    |
|  22       |  0.1688   |  9.514    |  30.37    |
|  23       |  0.009262 |  1.071    |  58.93    |
|  24       |  0.008939 |  1.0      |  172.9    |
|  25       |  0.007534 |  1.041    |  402.1    |
{'max_depth': 5, 'n_estimators': 392}
Sud_ S_
|   

|  25       |  0.7317   |  9.856    |  248.6    |
{'max_depth': 8, 'n_estimators': 358}
Ost_ O_
Sud_ S_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.7994   |  6.51     |  109.5    |
|  2        |  0.7894   |  4.925    |  391.6    |
|  3        |  0.7727   |  3.658    |  100.1    |
|  4        |  0.5187   |  1.202    |  227.5    |
|  5        |  0.7405   |  3.148    |  188.7    |
|  6        |  0.7962   |  9.916    |  141.7    |
|  7        |  0.6617   |  1.731    |  344.7    |
|  8        |  0.8      |  6.591    |  158.9    |
|  9        |  0.7888   |  5.196    |  85.63    |
|  10       |  0.6615   |  1.666    |  453.4    |
|  11       |  0.7997   |  8.146    |  425.1    |
|  12       |  0.7998   |  8.337    |  495.7    |
|  13       |  0.7976   |  6.195    |  412.5    |
|  14       |  0.7875   |  4.792    |  42.9     |
|  15       |  0.7882   |  5.087    |  79.5     |
|  16       |  0.7997   |  8.355    |  357.9  

|  15       |  0.4155   |  5.087    |  79.5     |
|  16       |  0.4174   |  8.355    |  357.9    |
|  17       |  0.4221   |  6.088    |  158.9    |
|  18       |  0.4056   |  9.986    |  94.88    |
|  19       |  0.4219   |  6.539    |  258.0    |
|  20       |  0.4147   |  4.645    |  372.1    |
|  21       |  0.4045   |  10.0     |  280.5    |
|  22       |  0.1427   |  1.415    |  301.8    |
|  23       |  0.144    |  1.0      |  125.6    |
|  24       |  0.4019   |  10.0     |  59.82    |
|  25       |  0.2746   |  1.976    |  270.1    |
{'max_depth': 7, 'n_estimators': 109}
Nord_ N_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.6686   |  6.51     |  109.5    |
|  2        |  0.6507   |  4.925    |  391.6    |
|  3        |  0.6278   |  3.658    |  100.1    |
|  4        |  0.4347   |  1.202    |  227.5    |
|  5        |  0.5892   |  3.148    |  188.7    |
|  6        |  0.6769   |  9.916    |  141.7    |
|  

...

|  1        |  0.8064   |  6.51     |  109.5    |
|  2        |  0.7992   |  4.925    |  391.6    |
|  3        |  0.7841   |  3.658    |  100.1    |
|  4        |  0.5952   |  1.202    |  227.5    |
|  5        |  0.7563   |  3.148    |  188.7    |
|  6        |  0.8084   |  9.916    |  141.7    |
|  7        |  0.7011   |  1.731    |  344.7    |
|  8        |  0.8067   |  6.591    |  158.9    |
|  9        |  0.7983   |  5.196    |  85.63    |
|  10       |  0.702    |  1.666    |  453.4    |
|  11       |  0.8088   |  8.146    |  425.1    |
|  12       |  0.8077   |  8.337    |  495.7    |
|  13       |  0.8043   |  6.195    |  412.5    |
|  14       |  0.7977   |  4.792    |  42.9     |
|  15       |  0.7997   |  5.087    |  79.5     |
|  16       |  0.8083   |  8.355    |  357.9    |
|  17       |  0.8038   |  6.088    |  158.9    |
|  18       |  0.8066   |  9.986    |  94.88    |
|  19       |  0.8063   |  6.539    |  258.0    |
|  20       |  0.7991   |  4.645    |  372.1    |


|  18       |  0.7026   |  9.986    |  94.88    |
|  19       |  0.701    |  6.539    |  258.0    |
|  20       |  0.6843   |  4.645    |  372.1    |
|  21       |  0.7009   |  9.877    |  124.4    |
|  22       |  0.7036   |  9.761    |  276.4    |
|  23       |  0.5717   |  1.606    |  295.3    |
|  24       |  0.7004   |  10.0     |  60.76    |
|  25       |  0.4549   |  1.0      |  482.6    |
{'max_depth': 8, 'n_estimators': 425}
West_ W_
NO2 rf_6_
DonBosco_ D_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.4198   |  6.51     |  109.5    |
|  2        |  0.4219   |  4.925    |  391.6    |
|  3        |  0.4055   |  3.658    |  100.1    |
|  4        |  0.1477   |  1.202    |  227.5    |
|  5        |  0.369    |  3.148    |  188.7    |
|  6        |  0.4022   |  9.916    |  141.7    |
|  7        |  0.3222   |  1.731    |  344.7    |
|  8        |  0.422    |  6.591    |  158.9    |
|  9        |  0.4205   |  5.1

|  8        |  0.7056   |  6.591    |  158.9    |
|  9        |  0.6646   |  5.196    |  85.63    |
|  10       |  0.497    |  1.666    |  453.4    |
|  11       |  0.7133   |  8.146    |  425.1    |
|  12       |  0.7122   |  8.337    |  495.7    |
|  13       |  0.6915   |  6.195    |  412.5    |
|  14       |  0.6648   |  4.792    |  42.9     |
|  15       |  0.6663   |  5.087    |  79.5     |
|  16       |  0.7118   |  8.355    |  357.9    |
|  17       |  0.6899   |  6.088    |  158.9    |
|  18       |  0.7129   |  9.986    |  94.88    |
|  19       |  0.7071   |  6.539    |  258.0    |
|  20       |  0.666    |  4.645    |  372.1    |
|  21       |  0.7134   |  10.0     |  154.3    |
|  22       |  0.7132   |  9.974    |  164.9    |
|  23       |  0.3476   |  1.095    |  421.1    |
|  24       |  0.7134   |  10.0     |  430.1    |
|  25       |  0.691    |  6.49     |  147.5    |
{'max_depth': 10, 'n_estimators': 154}
West_ W_
|   iter    |  target   | max_depth | n_esti... |
--

Ost_ O_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.2482   |  6.51     |  109.5    |
|  2        |  0.2376   |  4.925    |  391.6    |
|  3        |  0.224    |  3.658    |  100.1    |
|  4        |  0.00261  |  1.202    |  227.5    |
|  5        |  0.1912   |  3.148    |  188.7    |
|  6        |  0.2536   |  9.916    |  141.7    |
|  7        |  0.1182   |  1.731    |  344.7    |
|  8        |  0.2589   |  6.591    |  158.9    |
|  9        |  0.2481   |  5.196    |  85.63    |
|  10       |  0.1165   |  1.666    |  453.4    |
|  11       |  0.2611   |  8.146    |  425.1    |
|  12       |  0.264    |  8.337    |  495.7    |
|  13       |  0.2526   |  6.195    |  412.5    |
|  14       |  0.2304   |  4.792    |  42.9     |
|  15       |  0.2311   |  5.087    |  79.5     |
|  16       |  0.255    |  8.355    |  357.9    |
|  17       |  0.2476   |  6.088    |  158.9    |
|  18       |  0.2518   |  9.986    |  94.

|  17       |  0.8032   |  6.088    |  158.9    |
|  18       |  0.8062   |  9.986    |  94.88    |
|  19       |  0.8063   |  6.539    |  258.0    |
|  20       |  0.7987   |  4.645    |  372.1    |
|  21       |  0.8075   |  10.0     |  279.2    |
|  22       |  0.5951   |  1.263    |  300.8    |
|  23       |  0.806    |  10.0     |  59.68    |
|  24       |  0.5951   |  1.0      |  126.0    |
|  25       |  0.7016   |  1.976    |  270.1    |
{'max_depth': 8, 'n_estimators': 358}
Ost_ O_
Sud_ S_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.8521   |  6.51     |  109.5    |
|  2        |  0.8393   |  4.925    |  391.6    |
|  3        |  0.825    |  3.658    |  100.1    |
|  4        |  0.6013   |  1.202    |  227.5    |
|  5        |  0.7943   |  3.148    |  188.7    |
|  6        |  0.8551   |  9.916    |  141.7    |
|  7        |  0.7363   |  1.731    |  344.7    |
|  8        |  0.8527   |  6.591    |  158.9  

In [18]:
df = my_f()
df.to_csv(f'../output_intermediate/ML_Optimization/rf/rf_s1_{percent}.csv')

percent: 0.8
NO2 rf_base_
DonBosco_ D_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.163    |  6.51     |  109.5    |
|  2        |  0.1731   |  4.925    |  391.6    |
|  3        |  0.1747   |  3.658    |  100.1    |
|  4        |  0.1163   |  1.202    |  227.5    |
|  5        |  0.1659   |  3.148    |  188.7    |
|  6        |  0.1402   |  9.916    |  141.7    |
|  7        |  0.1425   |  1.731    |  344.7    |
|  8        |  0.1635   |  6.591    |  158.9    |
|  9        |  0.1709   |  5.196    |  85.63    |
|  10       |  0.1422   |  1.666    |  453.4    |
|  11       |  0.1588   |  8.146    |  425.1    |
|  12       |  0.1573   |  8.337    |  495.7    |
|  13       |  0.1684   |  6.195    |  412.5    |
|  14       |  0.1742   |  4.792    |  42.9     |
|  15       |  0.1711   |  5.087    |  79.5     |
|  16       |  0.1588   |  8.355    |  357.9    |
|  17       |  0.1695   |  6.088    |  158.9    |
|  18      

|  17       |  0.5897   |  6.088    |  158.9    |
|  18       |  0.5794   |  9.986    |  94.88    |
|  19       |  0.5915   |  6.539    |  258.0    |
|  20       |  0.5779   |  4.645    |  372.1    |
|  21       |  0.5798   |  10.0     |  280.4    |
|  22       |  0.3241   |  1.085    |  301.1    |
|  23       |  0.3253   |  1.0      |  126.6    |
|  24       |  0.5805   |  10.0     |  60.04    |
|  25       |  0.4591   |  1.976    |  270.1    |
{'max_depth': 7, 'n_estimators': 258}
West_ W_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.5825   |  6.51     |  109.5    |
|  2        |  0.5707   |  4.925    |  391.6    |
|  3        |  0.5514   |  3.658    |  100.1    |
|  4        |  0.3037   |  1.202    |  227.5    |
|  5        |  0.5231   |  3.148    |  188.7    |
|  6        |  0.5745   |  9.916    |  141.7    |
|  7        |  0.4548   |  1.731    |  344.7    |
|  8        |  0.5828   |  6.591    |  158.9    |
|  

|  7        |  0.07591  |  1.731    |  344.7    |
|  8        |  0.1307   |  6.591    |  158.9    |
|  9        |  0.1545   |  5.196    |  85.63    |
|  10       |  0.07444  |  1.666    |  453.4    |
|  11       |  0.1319   |  8.146    |  425.1    |
|  12       |  0.1289   |  8.337    |  495.7    |
|  13       |  0.1418   |  6.195    |  412.5    |
|  14       |  0.1391   |  4.792    |  42.9     |
|  15       |  0.1492   |  5.087    |  79.5     |
|  16       |  0.1297   |  8.355    |  357.9    |
|  17       |  0.1367   |  6.088    |  158.9    |
|  18       |  0.1143   |  9.986    |  94.88    |
|  19       |  0.1328   |  6.539    |  258.0    |
|  20       |  0.1516   |  4.645    |  372.1    |
|  21       | -0.05021  |  1.0      |  166.8    |
|  22       |  0.09956  |  10.0     |  82.75    |
|  23       | -0.05965  |  1.379    |  82.65    |
|  24       |  0.1297   |  6.565    |  87.99    |
|  25       |  0.1354   |  6.849    |  77.23    |
{'max_depth': 5, 'n_estimators': 86}
Sud_ S_
|   i

|  25       |  0.7234   |  8.454    |  125.5    |
{'max_depth': 7, 'n_estimators': 109}
Ost_ O_
Sud_ S_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.7962   |  6.51     |  109.5    |
|  2        |  0.7883   |  4.925    |  391.6    |
|  3        |  0.77     |  3.658    |  100.1    |
|  4        |  0.529    |  1.202    |  227.5    |
|  5        |  0.7401   |  3.148    |  188.7    |
|  6        |  0.7927   |  9.916    |  141.7    |
|  7        |  0.6667   |  1.731    |  344.7    |
|  8        |  0.7981   |  6.591    |  158.9    |
|  9        |  0.7883   |  5.196    |  85.63    |
|  10       |  0.6659   |  1.666    |  453.4    |
|  11       |  0.7972   |  8.146    |  425.1    |
|  12       |  0.7977   |  8.337    |  495.7    |
|  13       |  0.7957   |  6.195    |  412.5    |
|  14       |  0.7873   |  4.792    |  42.9     |
|  15       |  0.7873   |  5.087    |  79.5     |
|  16       |  0.7969   |  8.355    |  357.9  

|  15       |  0.4088   |  5.087    |  79.5     |
|  16       |  0.406    |  8.355    |  357.9    |
|  17       |  0.4125   |  6.088    |  158.9    |
|  18       |  0.3913   |  9.986    |  94.88    |
|  19       |  0.4116   |  6.539    |  258.0    |
|  20       |  0.4095   |  4.645    |  372.1    |
|  21       |  0.3929   |  10.0     |  281.0    |
|  22       |  0.3869   |  10.0     |  30.0     |
|  23       |  0.1237   |  1.0      |  126.1    |
|  24       |  0.3901   |  10.0     |  59.58    |
|  25       |  0.1208   |  1.492    |  300.1    |
{'max_depth': 6, 'n_estimators': 412}
Nord_ N_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.6727   |  6.51     |  109.5    |
|  2        |  0.6528   |  4.925    |  391.6    |
|  3        |  0.6265   |  3.658    |  100.1    |
|  4        |  0.4221   |  1.202    |  227.5    |
|  5        |  0.5875   |  3.148    |  188.7    |
|  6        |  0.6793   |  9.916    |  141.7    |
|  

...

|  2        |  0.7968   |  4.925    |  391.6    |
|  3        |  0.7821   |  3.658    |  100.1    |
|  4        |  0.5991   |  1.202    |  227.5    |
|  5        |  0.756    |  3.148    |  188.7    |
|  6        |  0.8032   |  9.916    |  141.7    |
|  7        |  0.7055   |  1.731    |  344.7    |
|  8        |  0.8022   |  6.591    |  158.9    |
|  9        |  0.7969   |  5.196    |  85.63    |
|  10       |  0.7047   |  1.666    |  453.4    |
|  11       |  0.8033   |  8.146    |  425.1    |
|  12       |  0.8036   |  8.337    |  495.7    |
|  13       |  0.8009   |  6.195    |  412.5    |
|  14       |  0.7966   |  4.792    |  42.9     |
|  15       |  0.7957   |  5.087    |  79.5     |
|  16       |  0.8038   |  8.355    |  357.9    |
|  17       |  0.8002   |  6.088    |  158.9    |
|  18       |  0.8023   |  9.986    |  94.88    |
|  19       |  0.8022   |  6.539    |  258.0    |
|  20       |  0.7968   |  4.645    |  372.1    |
|  21       |  0.8038   |  10.0     |  280.6    |


|  19       |  0.7005   |  6.539    |  258.0    |
|  20       |  0.6852   |  4.645    |  372.1    |
|  21       |  0.7014   |  10.0     |  276.7    |
|  22       |  0.469    |  1.0      |  125.7    |
|  23       |  0.4691   |  1.0      |  269.0    |
|  24       |  0.7033   |  9.949    |  150.7    |
|  25       |  0.7009   |  9.755    |  250.1    |
{'max_depth': 10, 'n_estimators': 151}
West_ W_
NO2 rf_6_
DonBosco_ D_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.4076   |  6.51     |  109.5    |
|  2        |  0.4112   |  4.925    |  391.6    |
|  3        |  0.4007   |  3.658    |  100.1    |
|  4        |  0.1483   |  1.202    |  227.5    |
|  5        |  0.3673   |  3.148    |  188.7    |
|  6        |  0.3883   |  9.916    |  141.7    |
|  7        |  0.3217   |  1.731    |  344.7    |
|  8        |  0.4093   |  6.591    |  158.9    |
|  9        |  0.4111   |  5.196    |  85.63    |
|  10       |  0.3209   |  1.

|  9        |  0.6612   |  5.196    |  85.63    |
|  10       |  0.4856   |  1.666    |  453.4    |
|  11       |  0.706    |  8.146    |  425.1    |
|  12       |  0.706    |  8.337    |  495.7    |
|  13       |  0.6889   |  6.195    |  412.5    |
|  14       |  0.6607   |  4.792    |  42.9     |
|  15       |  0.6628   |  5.087    |  79.5     |
|  16       |  0.7059   |  8.355    |  357.9    |
|  17       |  0.6877   |  6.088    |  158.9    |
|  18       |  0.7069   |  9.986    |  94.88    |
|  19       |  0.7017   |  6.539    |  258.0    |
|  20       |  0.6638   |  4.645    |  372.1    |
|  21       |  0.7064   |  9.773    |  152.1    |
|  22       |  0.7066   |  9.993    |  167.3    |
|  23       |  0.7058   |  9.61     |  124.3    |
|  24       |  0.7073   |  8.882    |  272.7    |
|  25       |  0.4857   |  1.561    |  285.8    |
{'max_depth': 9, 'n_estimators': 273}
West_ W_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  

Ost_ O_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.2441   |  6.51     |  109.5    |
|  2        |  0.2294   |  4.925    |  391.6    |
|  3        |  0.202    |  3.658    |  100.1    |
|  4        | -0.05168  |  1.202    |  227.5    |
|  5        |  0.1667   |  3.148    |  188.7    |
|  6        |  0.2413   |  9.916    |  141.7    |
|  7        |  0.08098  |  1.731    |  344.7    |
|  8        |  0.2503   |  6.591    |  158.9    |
|  9        |  0.2301   |  5.196    |  85.63    |
|  10       |  0.07905  |  1.666    |  453.4    |
|  11       |  0.2468   |  8.146    |  425.1    |
|  12       |  0.255    |  8.337    |  495.7    |
|  13       |  0.2454   |  6.195    |  412.5    |
|  14       |  0.2253   |  4.792    |  42.9     |
|  15       |  0.2373   |  5.087    |  79.5     |
|  16       |  0.2548   |  8.355    |  357.9    |
|  17       |  0.2451   |  6.088    |  158.9    |
|  18       |  0.2433   |  9.986    |  94.

|  17       |  0.7989   |  6.088    |  158.9    |
|  18       |  0.8009   |  9.986    |  94.88    |
|  19       |  0.8026   |  6.539    |  258.0    |
|  20       |  0.7967   |  4.645    |  372.1    |
|  21       |  0.8026   |  10.0     |  280.4    |
|  22       |  0.5989   |  1.085    |  301.1    |
|  23       |  0.5989   |  1.084    |  269.9    |
|  24       |  0.8032   |  10.0     |  249.1    |
|  25       |  0.8026   |  8.465    |  125.9    |
{'max_depth': 10, 'n_estimators': 142}
Ost_ O_
Sud_ S_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.8542   |  6.51     |  109.5    |
|  2        |  0.8422   |  4.925    |  391.6    |
|  3        |  0.8262   |  3.658    |  100.1    |
|  4        |  0.6073   |  1.202    |  227.5    |
|  5        |  0.7975   |  3.148    |  188.7    |
|  6        |  0.8562   |  9.916    |  141.7    |
|  7        |  0.7424   |  1.731    |  344.7    |
|  8        |  0.8545   |  6.591    |  158.9 